**Sentence Ordering**

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import nltk.data
import random
import re
import sklearn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Download and using the NLTK Punkt Tokenizer

nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# 5 different Permutations of the Summary and capping the number of sentences considerd to 10

SHUFFLE_COUNT = 5
MAX_SENTENCE_LENGTH = 10

In [ ]:
# Preprocessing Data

os.makedirs("./arxiv_data/original/", exist_ok=True)
os.makedirs("./arxiv_data/permutations/", exist_ok=True)
os.makedirs("./arxiv_data/processed/", exist_ok=True)

In [ ]:
# Reading ARXIV Data

with open("./arxivData.json", "r") as f:
    data = json.load(f)

In [ ]:
# Data Preprocessing

for index,summary in enumerate(data):    
    
    # Creating the file to store the correct ordering
    file_name = "./arxiv_data/original/original_" + str(index) + ".csv"
    f = open('%s' % file_name, 'w+')
    
    # Text Cleaning
    abstract = " ".join(summary['summary'].strip())
    re.sub('[^A-Za-z0-9]+', '', abstract)
    tokenized = tokenizer.tokenize(abstract)

    processed_sent = list(map(lambda st: str.replace(st, "   ", "<><>"), tokenized))
    tokenized = list(map(lambda st: str.replace(st, " ", ""), processed_sent))
    tokenized = list(map(lambda st: str.replace(st, "<><>", " "), tokenized))

    for line_no,sent in enumerate(tokenized):
        f.write(str(line_no) + ">> " + sent + "\n")
    
    f.close()

In [ ]:
# Generating Permutations

for index,summary in enumerate(data):
    
    file_name = "./arxiv_data/original/original_" + str(index) + ".csv"
    f = open('%s' % file_name, 'r')
    processed_data = f.readlines()
    
    for i in range(SHUFFLE_COUNT):
        
        perm_fname = "./arxiv_data/permutations/original_" + str(index) +"_permutation_" + str(i) + ".csv"
        sf = open('%s' % perm_fname, 'w+')
        
        if(i == 0):
            shuffled =''.join(processed_data)
        else:
            random.shuffle(processed_data)
            shuffled = ''.join(processed_data)
            
        sf.write(shuffled)
        sf.close()
    
    f.close()

In [ ]:
# Read the generated permutations

permutation_files =  os.listdir("./arxiv_data/permutations/")
permutations = []

for f in permutation_files:
    
    file = open("./arxiv_data/permutations/" + f)
    permutations.append(file.readlines())

**Vectorization**

In [ ]:
# Storing the original and permuted information [similar to one hot encoding]

org = np.array([[" " for i in range(MAX_SENTENCE_LENGTH)] for i in range(len(permutations))], 
                         dtype = object)
perm = np.array([[-1 for i in range(MAX_SENTENCE_LENGTH)] for i in range(len(permutations))])

for i in range(len(permutations)):
    
    info = permutations[i]
    j = 0

    for line in info:
        org[i][j] = ''.join(line.split('>> ')[1:]).strip()

        try:
            if(int(line.split('>> ')[0]) <= MAX_SENTENCE_LENGTH):
                perm[i][j] = int(line.split('>> ')[0])
                j += 1
            if( j >= MAX_SENTENCE_LENGTH ):
                break

        except (RuntimeError, TypeError, NameError, ValueError):
            pass

In [ ]:
# Storing Sentece information

with open('./arxiv_data/processed/sentences.txt','w+') as f:
    for line in org:
        f.write('\t'.join(line).strip() + '\n')

In [ ]:
# Storing Permuation information

with open("./arxiv_data/processed/permutations.txt",'w+') as f:
    for line in perm:
        f.write('\t'.join(line.astype(str)).strip() + '\n')

In [ ]:
data_folder = './arxiv_data/processed/'
modelFileName = 'model.h5'

# Defining parameters for the training data
split_at = 1800
batch_size = 10
max_sent_len = 40
max_num_word = 2000
MAX_SENTENCE_LENGTH = 10
EMBEDDING_DIM = 300

In [ ]:
# Novelty: creation of one-hot encoding vectors

y = np.loadtxt(data_folder + 'permutations.txt', delimiter='\t', dtype=int)

with open(data_folder + 'sentences.txt', encoding='utf8') as f:
    sentences = f.readlines()

# Creating the one hot encoding for the multiclass labels
onehot = []
for y_ in y:
    oneword = []
    dummyVec = np.array([ 0 for i in range(MAX_SENTENCE_LENGTH + 1)])
    for yy_ in y_:
        dummyVec[yy_] = 1
        oneword.append(np.copy(dummyVec))
        dummyVec[yy_] = 0
    onehot.append(oneword)

onehot = np.asarray(onehot)

tokenizer_file_extra = ''

# Creating the paragraph permutations
para_permutation = []
for line in sentences:
    lineSents = line.split('\t')
    sent = [[" "] for i in range(MAX_SENTENCE_LENGTH)]
    j = 0
    for ls in lineSents:
        sent[j][0] = ls
        j += 1
    para_permutation.append(sent)

para_permutation = np.asarray(para_permutation)
    
# split into train and test data
x_train = para_permutation[:split_at]
x_test = para_permutation[split_at:]

y_train = onehot[:split_at]
y_test = onehot[split_at:]

validation_data = (x_test, y_test)

In [ ]:
y_test[0]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
x_test[0]

array([['The architecture shows'],
       ['This relationship is exploited to'],
       ['In this paper we present a novel Neural Network algorithm for conducting'],
       ['We introduce a neural\n'],
       [' '],
       [' '],
       [' '],
       [' '],
       [' '],
       [' ']], dtype='<U80')

**Model Training**

In [ ]:
import json
import pickle
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Lambda

In [ ]:
# Tensorflow compatability check

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [ ]:
# Elmo Model

elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True, 
name='elmo_module')

In [ ]:
# Testing BERT

# def UniversalEmbedding(x):
#     return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)

In [ ]:
# Testing ELMO

# embeddings_1 = elmo_model(
#     ["the cat is on the", "hat on mat"],
#     signature="default",
#     as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# Using ELMO sentence embedding
# Since ELMO has a word embedding, we average out the sentences initially and the comprehension next
# to feed it into the network

x_final_train = []
for para in x_train:
	sent = []
	for s in para:
		sent.append(s[0])

	embeddings = elmo_model(sent, signature="default", as_dict=True)["elmo"]

	mean_embedding = tf.math.reduce_mean(embeddings_1, axis=1, keepdims=False, name=None)

	x_final_train.append(mean_embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: ignored

In [ ]:
embeddings

<tf.Tensor 'elmo_module_1_apply_default_2/aggregation/mul_3:0' shape=(1800, 108, 1024) dtype=float32>

In [ ]:
type(embeddings)

tensorflow.python.framework.ops.Tensor

In [ ]:
embeddings.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'elmo_module_1_apply_default_2/aggregation/mul_3:0' shape=(1800, 108, 1024) dtype=float32>>

In [ ]:
# Vectorizing testing and training data

x_vectorized_comb  = np.concatenate((x_train.flatten(), x_test.flatten()), axis=None)
x_vectorized_train = x_train.flatten()
x_vectorized_test  = x_train.flatten()

In [ ]:
embeddings.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'elmo_module_1_apply_default_1/aggregation/mul_3:0' shape=(2050000, 19, 1024) dtype=float32>>

In [ ]:
feature_columns = [tf.feature_column.numeric_column('x', shape=x_train.shape[1:])]

In [ ]:
feature_columns

[NumericColumn(key='x', shape=(10, 1), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [ ]:
# Tensorflow feature colums

feature_columns = [tf.feature_column.numeric_column('x', shape=x_train.shape[1:])]

**LSTM**

Pre defined architecture for the network

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import Input, Dense, LSTM, Lambda, TimeDistributed, Bidirectional, Dropout, BatchNormalization
from keras import Model
import keras.backend as K

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
def BiLstm(max_num_sent):
    sent_seq = Input(shape=(max_num_sent, 1), name='sent_inp', dtype=tf.string)
    sent_lstm = Bidirectional(LSTM(100, return_sequences=True))(sent_seq)
    sent_lstm = Dropout(0.5)(sent_lstm)
    final_lstm = Bidirectional(LSTM(300, return_sequences=True))(sent_lstm)
    final_lstm = Dropout(0.5)(final_lstm)
    output_seq = TimeDistributed(Dense(max_num_sent + 1, activation='softmax'))(final_lstm) 
    model = Model(inputs = sent_seq, outputs=output_seq)
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
# Model initialization
model = BiLstm(MAX_SENTENCE_LENGTH)

In [ ]:
# Training the model

os.makedirs("./Models/", exist_ok=True)
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(x_train, y_train, epochs=20, batch_size=batch_size,
                    validation_data=validation_data, shuffle=True)

  model.save_weights('./Models/' + modelFileName )

**Model Evaluation**

In [ ]:
import scipy.stats as stats

In [ ]:
def calulateKendallTau(y_true, y_pred):
    
    y_true_indexed = np.argmax(y_true, axis=2)
    y_pred_indexed = np.argmax(y_pred, axis=2)
    
    corrected_y_pred_indexed = (y_true_indexed != max_seq_len) * y_pred_indexed
    corrected_y_pred_indexed[corrected_y_pred_indexed == 0] = max_seq_len
    
    kendal_tau_list = []
    for i in range(y_true_indexed.shape[0]):
        kendal_tau_list.append(stats.kendalltau(y_true_indexed[i],corrected_y_pred_indexed[i])[0])
    
    return np.mean(kendal_tau_list)    

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./Models/' + modelFileName)
    y_pred = model.predict(x_test)

In [ ]:
calulateKendallTau(y_test,y_pred)